In [1]:
# import necessary packages
import os
import cv2
import time
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout

In [2]:
img_size = 100
datadir = r'Data'    # root data directiory 
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['no_mask', 'mask']


In [3]:
# Define two empty list for contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))
            
            # resize the image
            resized_array = cv2.resize(img_arr, (img_size, img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
            
PreProcess()
cv2.destroyAllWindows()


  0%|          | 0/500 [00:00<?, ?it/s]

Data/no_mask


  6%|▌         | 28/500 [00:00<00:01, 276.60it/s]

Data/mask


100%|██████████| 500/500 [00:01<00:00, 281.28it/s]


In [4]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [5]:
# Convert and resize the data to a numpy array
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [6]:
batch_size = 32
epochs = 15

In [7]:
# Create the model architecture

model = Sequential()

model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))


model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))

# compile the model

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 256)       147712    
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 128)       2

In [8]:
t1 = time.time()

# fit the model
model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))

t2 = time.time()
print('Time taken: ',t2-t1)

Epoch 1/15
18/18 [==============================] - 14s 269ms/step - loss: 0.2033 - accuracy: 0.9000 - val_loss: 2.8310e-06 - val_accuracy: 1.0000
Epoch 2/15
18/18 [==============================] - 2s 132ms/step - loss: 0.0257 - accuracy: 0.9946 - val_loss: 0.0169 - val_accuracy: 0.9958
Epoch 3/15
18/18 [==============================] - 2s 142ms/step - loss: 0.0168 - accuracy: 0.9964 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 4/15
18/18 [==============================] - 2s 107ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 9.2997e-06 - val_accuracy: 1.0000
Epoch 5/15
18/18 [==============================] - 2s 93ms/step - loss: 0.0075 - accuracy: 0.9964 - val_loss: 6.9924e-06 - val_accuracy: 1.0000
Epoch 6/15
18/18 [==============================] - 3s 155ms/step - loss: 0.0218 - accuracy: 0.9929 - val_loss: 0.1932 - val_accuracy: 0.9375
Epoch 7/15
18/18 [==============================] - 1s 68ms/step - loss: 0.0521 - accuracy: 0.9804 - val_loss: 0.0098 - val_accuracy: 0.

In [9]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 67ms/step - loss: 3.1590e-08 - accuracy: 1.0000
